#  Credit Scoring with Snowpark for Python Set-up Notebook
Author: Zohar Nissare-Houssen

## 1. Snowflake Trial Account

The prerequisite is to have a Snowflake account. If you do not have a Snowflake account, you can sign-up for a free 30 day [Snowflake trial](https://signup.snowflake.com/).

After signing-up for the trial, please bookmark the URL of the Snowflake account, and save your credentials as they will be needed in this lab.


This version requires Snowpark **0.4.0** or higher

## 2. Python Libraries

The following libraries are needed to run this demo. In this section, add any python library missing in your environment.

In [1]:
pip install scikit-plot

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyarrow==6.0.0

     --------------------------------------- 15.5/15.5 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 8.0.0
    Uninstalling pyarrow-8.0.0:
      Successfully uninstalled pyarrow-8.0.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install seaborn

In [ ]:
pip install matplotlib

## 3. File Download

### 3.1 The Dataset

In [3]:
! curl -O https://raw.githubusercontent.com/zoharsan/snowpark_credit_score/main/credit_files.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  292k  100  292k    0     0   797k      0 --:--:-- --:--:-- --:--:--  801k


In [4]:
! curl -O https://raw.githubusercontent.com/zoharsan/snowpark_credit_score/main/credit_request.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  6068  100  6068    0     0  22385      0 --:--:-- --:--:-- --:--:-- 22474


### 3.2 The creds.json credential file

The file below needs to be edited with credentials of your Snowflake account and saved. It will be used to connect to Snowflake on the main Notebook:


```
{
  "account": "<account-name>",
  "user": "<user>",
  "password": "<password>",
  "warehouse": "<warehouse-name>",
  "database": "CREDIT_BANK",
  "schema": "PUBLIC"
}
```   

In [5]:
! curl -O https://github.com/zoharsan/snowpark_credit_score/raw/main/creds.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


## 4. The Database

In the section below, please fill-up the different parameters to connect to your Snowflake Environment in the cell below.

In [9]:
from snowflake.snowpark import *
from snowflake.snowpark import version
from snowflake.snowpark.functions import *

import pandas as pd
connection_parameters = {
    "account": "kb35457.eu-west-1",
    "user": "anamhuluba",
    "password": "M@maliga13",
    "warehouse": "COMPUTE_WH",
}

session = Session.builder.configs(connection_parameters).create()

session.sql("create or replace database credit_bank").collect()
session.sql("use schema credit_bank.public").collect()
print(session.sql("select current_warehouse(), current_database(), current_schema(), current_user(), current_role()").collect())

[Row(CURRENT_WAREHOUSE()='COMPUTE_WH', CURRENT_DATABASE()='CREDIT_BANK', CURRENT_SCHEMA()='PUBLIC', CURRENT_USER()='ANAMHULUBA', CURRENT_ROLE()='ACCOUNTADMIN')]


## 5. The Tables

There are 2 tables associated with this demo:

* CREDIT_FILES: This table contains currently the credit on files along with the credit standing whether the loan is being repaid or if there are actual issues with reimbursing the credit. This dataset is going to be used for historical analysis and build a machine learning model to score new applications.

* CREDIT_REQUESTS: This table contains the new credit requests that the bank needs to provide approval on based on the ML algorithm.


### 5.1 CREDIT_FILES Table



After check running the command below, log into your Snowflake environment and make sure the table was created. It should have 2.9K rows. DO NOT RUN THIS TWICE. Otherwise, it will append the rows twice making the ML model appear overfitting. If you need to rerun it, drop the table first (from the snowflake console or here following the syntax above eg ```session.sql("drop table CREDIT_FILES").collect()```

In [10]:
credit_files = pd.read_csv('credit_files.csv')
session.write_pandas(credit_files,"CREDIT_FILES",auto_create_table='True')

In [11]:
credit_df = session.table("CREDIT_FILES")
credit_df.schema

StructType([StructField('CREDIT_REQUEST_ID', LongType(), nullable=True), StructField('CREDIT_AMOUNT', LongType(), nullable=True), StructField('CREDIT_DURATION', LongType(), nullable=True), StructField('PURPOSE', StringType(), nullable=True), StructField('INSTALLMENT_COMMITMENT', LongType(), nullable=True), StructField('OTHER_PARTIES', StringType(), nullable=True), StructField('CREDIT_STANDING', StringType(), nullable=True), StructField('CREDIT_SCORE', LongType(), nullable=True), StructField('CHECKING_BALANCE', DoubleType(), nullable=True), StructField('SAVINGS_BALANCE', DoubleType(), nullable=True), StructField('EXISTING_CREDITS', LongType(), nullable=True), StructField('ASSETS', StringType(), nullable=True), StructField('HOUSING', StringType(), nullable=True), StructField('QUALIFICATION', StringType(), nullable=True), StructField('JOB_HISTORY', LongType(), nullable=True), StructField('AGE', LongType(), nullable=True), StructField('SEX', StringType(), nullable=True), StructField('MARIT

In [27]:
session.sql('show tables').show()
credit_df.show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"        |"database_name"  |"schema_name"  |"kind"  |"comment"  |"cluster_by"  |"rows"  |"bytes"  |"owner"       |"retention_time"  |"automatic_clustering"  |"change_tracking"  |"search_optimization"  |"search_optimization_progress"  |"search_optimization_bytes"  |"is_external"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2022-09-04 14:39:53.727000-07:

In [29]:
import pandas as pd
pds = credit_df.toPandas()

SnowparkFetchDataException: (1406): Failed to fetch a Pandas Dataframe. The error is: to_pandas() did not return a Pandas DataFrame. If you use session.sql(...).to_pandas(), the input query can only be a SELECT statement. Or you can use session.sql(...).collect() to get a list of Row objects for a non-SELECT statement, then convert it to a Pandas DataFrame.

### 5.2 CREDIT_REQUEST Table

After check running the command below, log into your Snowflake environment and make sure the table was created. It should have 60 rows. DO NOT RUN THIS TWICE. Otherwise, it will append the rows twice If you need to rerun it, drop the table first (from the snowflake console or here following the syntax above eg ```session.sql("drop table CREDIT_REQUESTS").collect()```

In [30]:
credit_requests = pd.read_csv('credit_request.csv')
session.write_pandas(credit_requests,"CREDIT_REQUESTS",auto_create_table='True')

In [31]:
credit_req_df = session.table("CREDIT_REQUESTS")
credit_req_df.schema

StructType([StructField('CREDIT_REQUEST_ID', LongType(), nullable=True), StructField('CREDIT_AMOUNT', LongType(), nullable=True), StructField('CREDIT_DURATION', LongType(), nullable=True), StructField('PURPOSE', StringType(), nullable=True), StructField('INSTALLMENT_COMMITMENT', LongType(), nullable=True), StructField('OTHER_PARTIES', StringType(), nullable=True), StructField('CREDIT_SCORE', LongType(), nullable=True), StructField('CHECKING_BALANCE', DoubleType(), nullable=True), StructField('SAVINGS_BALANCE', DoubleType(), nullable=True), StructField('EXISTING_CREDITS', LongType(), nullable=True), StructField('ASSETS', StringType(), nullable=True), StructField('HOUSING', StringType(), nullable=True), StructField('QUALIFICATION', StringType(), nullable=True), StructField('JOB_HISTORY', LongType(), nullable=True), StructField('AGE', LongType(), nullable=True), StructField('SEX', StringType(), nullable=True), StructField('MARITAL_STATUS', StringType(), nullable=True), StructField('NUM_DE

In [32]:
credit_req_df.toPandas().head()

SnowparkFetchDataException: (1406): Failed to fetch a Pandas Dataframe. The error is: to_pandas() did not return a Pandas DataFrame. If you use session.sql(...).to_pandas(), the input query can only be a SELECT statement. Or you can use session.sql(...).collect() to get a list of Row objects for a non-SELECT statement, then convert it to a Pandas DataFrame.

In [33]:
credit_req_df.toPandas().info()

SnowparkFetchDataException: (1406): Failed to fetch a Pandas Dataframe. The error is: to_pandas() did not return a Pandas DataFrame. If you use session.sql(...).to_pandas(), the input query can only be a SELECT statement. Or you can use session.sql(...).collect() to get a list of Row objects for a non-SELECT statement, then convert it to a Pandas DataFrame.